The first step is installing the Dash framework, including its core components (dash, dash-core-components, dash-html-components, and dash-table).

In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


## 1.Data Loading and Preparation

Dataset: The code starts by loading an Excel file containing financial data. The file is read using pandas.read_excel() into a DataFrame (df).

Dropping Columns: The "Submission Date" column, if present, is dropped using the errors='ignore' parameter, ensuring no errors occur even if the column is missing.

Months Definition: A list of months (January to December) is defined for iterating through the dataset.

Monthly Totals Calculation:The program calculates the average expenses for each month across relevant columns.
Columns containing the word "Income" are excluded from expense calculations.
The totals are rounded to integers and stored in the monthly_totals dictionary.

Detailed Monthly Expenses:
The monthly_expenses dictionary stores breakdowns of each month’s expenses by individual columns, allowing a more granular analysis of spending patterns.

## 2.Bar Chart for Monthly Totals

A bar chart is created using Plotly to display total monthly expenses.

Dynamic Bar Opacity: Opacity is normalized based on expenditure values, making higher expenses appear more prominent visually.
The normalized_opacity variable adjusts bar colors dynamically between 30% and 100% opacity, depending on the total value.

Interactivity:Bars include metadata (customdata) for the corresponding month, enabling interaction between the chart and other components of the app (e.g., the Sankey diagram).

Bar Styling:Bars are colored in a gradient shade (based on rgba) and have black borders for visual clarity.

Hover Information:Each bar shows a tooltip displaying the month and total expense when hovered over.

##3.Sankey Diagram for Income and Expense Breakdown

When a user clicks on a month in the bar chart, the app dynamically generates a Sankey diagram for that month.

The Sankey diagram provides a layered breakdown of financial flows:

Income → Expense Categories → Subcategories/Savings.

Data Processing for Sankey Diagram:

- Income Extraction: The program identifies the relevant income column(s) for the selected month and calculates the average income.

- Expense Categories:Expenses are grouped into the following categories based on column labels:

  -Rent/Utilities: Expenses related to housing and utilities.

  -Groceries/Subscriptions/Travel/Miscellaneous: A broad category for day-to-day expenses.

  -Tax: Expenses related to tax payments.

  Subcategories within each category are also identified and summed up.

- Savings: The difference between income and total expenses is calculated as savings.

Building the Sankey Diagram:

Nodes:
Represent income, categories, subcategories, and savings.
Each node is labeled (e.g., "Income," "Rent/Utilities") and assigned a distinct color.

Links:
Represent the flow of values between nodes (e.g., income → categories, categories → subcategories).
Each link is colored to match the corresponding category, with lighter shades for subcategories.

Annotations:
A motivational annotation is added above the Sankey diagram:
If savings are positive, a green message ("Your savings are good") is displayed.
If savings are negative or zero, a red message ("Time to review your spending") is shown.

##4. Web Application Layout

The app layout consists of two main sections:

Monthly Bar Chart: Displays total expenses for each month. Users can click on a bar to select a month.

Sankey Diagram: Displays the income and expense breakdown for the selected month.

Both components are created using Dash’s html.Div and dcc.Graph elements.


##5. Interactivity with Callbacks

The interactivity is managed using Dash's callback functions.

Input: The callback listens for clickData from the bar chart.

Output: When a bar is clicked, the corresponding month’s data is used to generate the Sankey diagram.

If no month is selected, an empty Sankey diagram is displayed.


##6. Execution
The application is run locally with debug=True for development purposes. This allows real-time updates and easier debugging.



In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import dash
from dash import dcc, html, Input, Output

# Load your dataset (update the file path as needed)
file_path = 'Money_MindScape_Data.xlsx'
df = pd.read_excel(file_path)

# Drop the "Submission Date" column if it exists
df = df.drop(columns=["Submission Date"], errors='ignore')

# Define the months of the year
months = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

# Initialize monthly_totals before using it
monthly_totals = {}

# Iterate through each month to populate monthly_totals first
for month in months:
    monthly_expense = 0
    for col in df.columns:
        if month in col and "Income" not in col:
            monthly_expense += round(df[col].mean())  # Round the average to integer
    monthly_totals[month] = monthly_expense

monthly_expenses = {}
# Iterate through each month
for month in months:
    monthly_expense = 0
    for col in df.columns:
        if month in col and "Income" not in col:
            monthly_expense += round(df[col].mean())
    monthly_totals[month] = monthly_expense

    # Extract expenses for each month
    monthly_expenses[month] = {}
    for col in df.columns:
        if month in col and "Income" not in col:
            monthly_expenses[month][col] = round(df[col].mean())

# Normalize opacity for interactivity
values = np.array(list(monthly_totals.values()))
normalized_opacity = (values - values.min()) / (values.max() - values.min())

# Initialize Dash app
app = dash.Dash(__name__)

# Create bar chart for the monthly totals
def create_bar_chart():
    fig = go.Figure()

    for i, (month, value) in enumerate(monthly_totals.items()):
        fig.add_trace(go.Bar(
            x=[month],
            y=[value],
            marker_color=f'rgba(42, 157, 143, {0.3 + 0.7 * normalized_opacity[i]})',
            marker_line_color='black',
            marker_line_width=1.5,
            name=month,
            customdata=[month],
            hoverinfo='text',
            hovertext=f'{month}: Total {value}',
        ))

    fig.update_layout(
        title="Interactive Monthly Totals Dashboard",
        xaxis=dict(title="Months"),
        yaxis=dict(title="Total Expenditure"),
        clickmode='event+select',
        showlegend=False,
    )
    return fig

def create_multi_layer_sankey(month):
    if month not in monthly_expenses:
        return go.Figure()

    # Extract the income column for the month
    income_columns = [col for col in df.columns if 'Income' in col and month in col]
    if not income_columns:
        return go.Figure()

    # Calculate average income for the month
    income_value = df[income_columns].mean().sum()

    # Calculate total expenses for the month
    total_expense = sum(monthly_expenses[month].values())

    # Calculate savings
    savings_value = income_value - total_expense

    # Categorize expenses into layers
    categories = {
        "Rent/Utilities": [],
        "Groceries/Subscriptions/Travel/Miscellaneous": [],
        "Tax": []
    }

    for expense, value in monthly_expenses[month].items():
        label = expense.split('_')[0]
        avg_value = round(df[expense].mean())
        if any(x in label.lower() for x in ["rent", "utilities"]):
            categories["Rent/Utilities"].append((label, avg_value))
        elif any(x in label.lower() for x in ["groceries", "subscriptions", "travel", "miscellaneous"]):
            categories["Groceries/Subscriptions/Travel/Miscellaneous"].append((label, avg_value))
        elif "tax" in label.lower():
            categories["Tax"].append((label, avg_value))

    labels = [f"{month} Income"]
    sources, targets, values, link_colors = [], [], [], []

    category_colors = {
        "Rent/Utilities": "rgba(42, 157, 143, 0.8)",
        "Groceries/Subscriptions/Travel/Miscellaneous": "rgba(233, 196, 106, 0.8)",
        "Tax": "rgba(231, 111, 81, 0.8)",
        "Savings": "rgba(135, 206, 250, 0.6)"
    }

    for category, items in categories.items():
        if items:
            labels.append(category)
            sources.append(0)
            targets.append(len(labels) - 1)
            values.append(np.sum([value for _, value in items]))
            link_colors.append(category_colors.get(category, "rgba(0, 0, 0, 0.5)"))

    for category_index, (category, items) in enumerate(categories.items(), start=1):
        for label, value in items:
            labels.append(label)
            sources.append(category_index)
            targets.append(len(labels) - 1)
            values.append(value)
            parent_color = category_colors.get(category, "rgba(0, 0, 0, 0.5)")
            lighter_color = parent_color.replace("0.8", "0.4")  # Lighter shade
            link_colors.append(lighter_color)

    if savings_value > 0:
        labels.append("Savings")
        sources.append(0)
        targets.append(len(labels) - 1)
        values.append(savings_value)
        link_colors.append(category_colors["Savings"])

    node_colors = []
    first_level_colors = ["blue", "green", "orange", "red"]

    for i, label in enumerate(labels):
        if i == 0:
            node_colors.append(first_level_colors[0])
        elif label in categories.keys():
            color_index = list(categories.keys()).index(label)
            node_colors.append(first_level_colors[color_index + 1])
        else:
            parent_category_index = sources[i - 1]
            parent_color = node_colors[parent_category_index]
            lighter_color = parent_color.replace("0.4", "0.2")
            node_colors.append(lighter_color)

    sankey_fig = go.Figure(go.Sankey(
        node=dict(
            pad=30,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels,
            color=node_colors
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            color=link_colors
        )
    ))

    if savings_value > 0:
        message = "Your savings are good 🤑"
        message_color = "green"
    else:
        message = "Time to review your spending 😞"
        message_color = "red"

    sankey_fig.add_annotation(
        x=1, y=1.2,
        text=message,
        showarrow=False,
        font=dict(size=16, color=message_color),
        bgcolor="white",
        borderpad=4,
        bordercolor="black",
        borderwidth=1
    )

    sankey_fig.update_layout(
        title=f"Multi-Layer Income and Expenses Breakdown for {month}",
    )

    return sankey_fig

app.layout = html.Div([
    dcc.Graph(
        id='monthly-bar-chart',
        figure=create_bar_chart()
    ),
    html.Div([
        html.H2("Sankey Diagram"),
        dcc.Graph(id='sankey-diagram')
    ])
])

@app.callback(
    Output('sankey-diagram', 'figure'),
    [Input('monthly-bar-chart', 'clickData')]
)
def update_sankey(clickData):
    if clickData is None:
        return go.Figure()

    month = clickData['points'][0]['customdata']
    return create_multi_layer_sankey(month)

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>